In [81]:
import d2d
import importlib
importlib.reload(d2d)
import numpy as np
import tensorflow as tf
import pandas as pd


In [82]:
linear, lstm_model, dnn_model, df_all_chan,input_columns  = d2d.import_data()

da = df_all_chan.to_numpy()

# chan_mean = da[:,0:2495].flatten().mean()
# chan_std = da[:,0:2495].flatten().std()
# discharge_mean = da[:,2496].flatten().mean()
# discharge_std = da[:,2496].flatten().std()

# da[:,0:2495] = (da[:,0:2495] - chan_mean) / chan_std
# da[:,2496] = (da[:,2496] - discharge_mean) / discharge_std
df_all_chan

Keys: <KeysViewHDF5 ['DAS Data', 'Discharge', 'Times']>


,0,1,2,3,4,5,6,7,8,9,...,2487,2488,2489,2490,2491,2492,2493,2494,2495,Discharge
0,1737.318436,1932.321258,1788.269101,3706.977556,1831.756626,1768.674227,1908.850695,3690.341997,1804.145150,2437.436803,...,3603.932847,2066.070800,2215.669390,1881.772352,3990.883595,1584.965970,1787.281257,1862.912440,4160.545583,9.947179
1,8161.349279,8245.783723,8118.189935,528.834231,468.890511,714.714310,728.051775,158.440806,82.719811,116.949415,...,8238.452283,8204.901437,8194.671705,8228.769841,8211.066134,8229.370787,8218.436955,8210.143402,8178.032001,9.934657
2,8242.942236,8205.640252,8093.980026,510.832905,451.884623,709.510655,709.932307,153.715844,83.404382,116.112236,...,8184.500407,8221.490571,8202.429647,8194.927561,8197.675324,8188.718827,8189.658071,8225.017259,8163.918322,9.929290
3,8159.780047,8196.654224,8113.873839,489.734947,438.657688,690.418208,701.896139,158.058233,83.003254,112.787627,...,8173.860868,8156.187127,8151.113841,8235.843141,8145.360321,8192.392266,8169.275389,8129.926491,8161.652382,9.923923
4,8198.498465,8161.470921,8120.452338,521.425978,444.272741,689.710217,695.164917,160.246640,84.054029,111.125592,...,8198.827286,8234.815465,8219.987333,8140.961605,8193.517740,8157.154485,8241.648596,8229.194191,8234.212679,9.918557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62740,8250.132344,8239.956853,8120.606117,709.708516,524.745767,837.184738,489.648388,74.456188,99.460249,91.911794,...,8237.995892,8243.180597,8263.685370,8236.631204,8275.920631,8158.055021,8254.067590,8260.170060,8189.577468,9.264333
62741,8305.046581,8215.717055,8123.947165,713.983575,522.460494,877.141941,504.369225,73.276914,98.030764,95.958327,...,8244.600296,8220.651129,8233.808688,8202.925678,8272.382131,8208.732997,8185.700921,8199.439315,8194.036074,9.263708
62742,8233.246034,8212.890322,8126.486773,712.414299,520.468924,892.210041,495.762205,72.542053,97.112611,98.886336,...,8296.184469,8233.850012,8214.919239,8173.153815,8231.181168,8183.252249,8171.110881,8196.319138,8220.779280,9.263083
62743,8230.088927,8237.590610,8188.620285,705.487651,523.552127,904.283215,506.508453,72.315307,96.342666,101.163440,...,8232.138229,8214.366628,8275.731039,8266.643805,8154.573206,8208.317212,8251.329151,8267.316168,8207.721162,9.262458


In [83]:
# test_data_frame = pd.DataFrame({'0':range(0,10000), 'Discharge':range(0,10000)})
# input_columns = ['0']

In [84]:
# print('Number of DAS observations: %d'%df_all_chan.shape[0])
# window_input_width = 200
# print('Number of tf Dataset windows: %d'%np.floor(df_all_chan.shape[0]/window_input_width))

In [85]:


# multi_step_window = d2d.WindowGenerator(test_data_frame,
#     input_width=window_input_width, label_width=1, shift=0,
#     label_columns=['Discharge'],
#     input_columns=input_columns,
#     shuffle=False)
window_input_width = 200
multi_step_window_shuffled = d2d.WindowGenerator(df_all_chan,
    input_width=window_input_width, label_width=1, shift=0,
    label_columns=['Discharge'],
    input_columns=input_columns,
    shuffle=True)

783
[8208.589  8212.292  8182.0913 ...  622.7094  820.1865 1272.743 ]


In [8]:
# ntest = tf.data.experimental.cardinality(multi_step_window.test)
# nval = multi_step_window.val.cardinality().numpy()
# ntrain = multi_step_window.train.cardinality().numpy()
# print('Size of test ds: %d'%ntest)
# print('Size of val ds: %d'%nval)
# print('Size of train ds: %d'%ntrain)
# print('Sum of ds sizes = %d'%(ntest+nval+ntrain))

In [10]:
multi_step_window_shuffled.example

(<tf.Tensor: shape=(64, 200, 2308), dtype=float32, numpy=
 array([[[ 2.631115  ,  2.6482012 ,  2.626265  , ..., -0.35355526,
          -0.37725192, -0.28030857],
         [ 2.6433623 ,  2.6431003 ,  2.6124547 , ..., -0.34071317,
          -0.3811673 , -0.28173247],
         [ 2.6264868 ,  2.618545  ,  2.62869   , ..., -0.31520796,
          -0.38014078, -0.29988238],
         ...,
         [ 2.630271  ,  2.6225283 ,  2.6288824 , ..., -0.4389694 ,
          -0.38926747, -0.3060913 ],
         [ 2.6478782 ,  2.6301763 ,  2.6336417 , ..., -0.4434537 ,
          -0.3665741 , -0.28363022],
         [ 2.6154823 ,  2.6264093 ,  2.6224155 , ..., -0.4536611 ,
          -0.36352977, -0.30339795]],
 
        [[ 2.6376789 ,  2.6478684 ,  2.6250632 , ..., -0.4291839 ,
          -0.34950936, -0.1537121 ],
         [ 2.6436574 ,  2.6497772 ,  2.5949    , ..., -0.4582502 ,
          -0.31459904, -0.11148059],
         [ 2.6452603 ,  2.6313171 ,  2.6261768 , ..., -0.46988043,
          -0.33612028, -0.

In [ ]:
history = d2d.compile_and_fit(linear, multi_step_window, learning_rate = 0.001)

In [ ]:
val_performance = this_model.evaluate(multi_step_window.val)
performance = this_model.evaluate(multi_step_window.test, verbose=0)
